In [12]:
# %load_ext dotenv
# %dotenv ../.env
# %load_ext autoreload

In [1]:
import os
from typing import Annotated
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_react_agent
from langchain_openai import OpenAI
from langchain import hub

In [38]:
llm = ChatOpenAI(  
    model="gpt-4o",  
    temperature=0,  
    max_tokens=None,  
    timeout=None,  
    max_retries=2,
)

In [9]:
llm = ChatOpenAI(
    api_key="ollama",
    model="gemma2:27b",
    base_url="http://localhost:11434/v1",
    temperature=0,
)

In [10]:
llm.invoke("hello")

AIMessage(content='Hello! 👋\n\nHow can I help you today? 😊', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 10, 'total_tokens': 23}, 'model_name': 'gemma2:27b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-7b8b768a-30ec-4ad6-9baf-307eb44f7324-0', usage_metadata={'input_tokens': 10, 'output_tokens': 13, 'total_tokens': 23})

In [17]:
from langchain.tools import tool
from typing import Optional, Type
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import BaseTool
from langchain.pydantic_v1 import BaseModel, Field
from langchain.callbacks.manager import CallbackManagerForToolRun
from langchain_core.output_parsers import StrOutputParser

class AnswerAboutSelfInput(BaseModel):
    query: str = Field(description="query about yourself or user greeting")

class AnswerAboutSelf(BaseTool):
    name = "answer_about_self"
    description = "Use this tool if the user query is a question about the chatbot, or if it is a greeting or a farewell.\nThe action input must be the user query unedited."
    args_schema: Type[BaseModel] = AnswerAboutSelfInput
    return_direct = True

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        system_prompt = (
            "Your name is Hoku. You are an assistant for answering questions about UH Manoa.\n"
            "You were initially created during the Hawaii Annual Code Challenge by team DarkMode.\n"
            "You are currently under development.\n"
            "Only respond with information given here.\n"
            "Answer concisely do not add any additional information"
        )

        qa_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{query}"),
            ]
        )

        chain = qa_prompt | self.metadata['llm'] | StrOutputParser()
        return chain.invoke({"query": query})

In [ ]:
class ManoaNewsInput(BaseModel):
    query: str = Field(description="query about current events at UH Manoa")

class ManoaNews(BaseTool):
    name = "answer_about_self"
    description = "Use this tool if asked about current events or questions that need real-time data related to UH Manoa."
    args_schema: Type[BaseModel] = ManoaNewsInput
    return_direct = False

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        system_prompt = (
            "Your name is Hoku. You are an assistant for answering questions about UH Manoa.\n"
            "You were initially created during the Hawaii Annual Code Challenge by team DarkMode.\n"
            "You are currently under development.\n"
            "Only respond with information given here.\n"
            "Answer concisely do not add any additional information"
        )

        qa_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{query}"),
            ]
        )

        chain = qa_prompt | self.metadata['llm'] | StrOutputParser()
        return chain.invoke({"query": query})
    


In [ ]:
tools = [AnswerAboutSelf(metadata={'llm': llm})]
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [48]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [21]:
agent_executor.invoke({"input": "hello"})



> Entering new AgentExecutor chain...
Thought: This is a greeting.
Action: answer_about_self
Action Input: helloHello!  I am Hoku, your UH Manoa assistant.


> Finished chain.


{'input': 'hello', 'output': 'Hello!  I am Hoku, your UH Manoa assistant.'}